In [1]:
using Pkg

Pkg.activate("..")
using Revise

using PeriDyn
using PDMesh

using Random
using Distributed, Folds


  Activating environment at `/mnt/D/git_repos/PeriDyn/Project.toml`
┌ Info: Precompiling PeriDyn [c3db0ce0-6d7e-41b5-be11-bcb6aafada44]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **



In [2]:
PeriDyn.TIMEIT_REF[] = true
Random.seed!(42)

resolution = 1


1

In [3]:
x1, v1, y1, vol1, type1 = create(Cuboid([-50 50; -50 50; 0 5]), resolution=resolution, type=1)

v1[1,:] .= 0.01

hor1 = 3.0
mat_gen1 = GeneralMaterial(y1, v1, x1, vol1, type1, hor1, max_neigh=200)

K, G = 1.0, 1.0
den1 = 1000.0
cstretch1 = 0.5
mat_spec1 = OrdinaryStateBasedSpecific([K], [G], [cstretch1], [den1])
# mat_spec1 = SkipSpecific([den1])

block1 = PeridynamicsMaterial(mat_gen1, mat_spec1; name="block 1")

Average family members: 71.66328


OrdinaryStateBasedMaterial("block 1", 1:1, GeneralMaterial
type: [1]
horizon: 3.0
particle size: [1.0]
, OrdinaryStateBasedSpecific
bulk_modulus: [1.0]
shear_modulus: [1.0]
critical_stretch: [0.5]
density: [1000.0]
)

In [4]:
x2, v2, y2, vol2, type2 = create(move(Cuboid([-5 5; -5 5; -5 5]), by=[0, 0.0, 15.0]), resolution=resolution, type=2)

v2[3,:] .-= 0.01

hor2 = 3.0
mat_gen2 = GeneralMaterial(y2, v2, x2, vol2, type2, hor2, max_neigh=200)

K2, G2 = 1.0, 1.0
den2 = 10000.0
cstretch2 = 0.5
mat_spec2 = OrdinaryStateBasedSpecific([K2], [G2], [cstretch2], [den2])
# mat_spec2 = SkipSpecific([den2])

block2 = PeridynamicsMaterial(mat_gen2, mat_spec2; name="block 2")

Average family members: 66.264


OrdinaryStateBasedMaterial("block 2", 2:2, GeneralMaterial
type: [2]
horizon: 3.0
particle size: [1.0]
, OrdinaryStateBasedSpecific
bulk_modulus: [1.0]
shear_modulus: [1.0]
critical_stretch: [0.5]
density: [10000.0]
)

In [5]:
k = 1000.0
blocks = [block1, block2]

RM = [
        [LinearRepulsionModel(k, b1, distanceX=2, max_neighs=200) for b1 in blocks]..., 
        [LinearRepulsionModel(k, blocks[i], blocks[j], distanceX=2, max_neighs=200) for i in 2:length(blocks) for j in 1:i-1]...
    ]

env = Env(1, blocks, RM, [], 0.5)

nothing
#




In [7]:
velocity_verlet!([env], 5000, filewrite_freq=10, neigh_update_freq=10, file_prefix="2blocks", start_at=0)


Updating neighbors for collision..................
Average repulsive neighs: 0.0
Average repulsive neighs: 0.0
No repulsive neighs.
Done
Material force calculation passed (block 1). [Undeformed]
Material force calculation passed (block 2). [Undeformed]
Material force calculation passed (block 1). [Undeformed]
Material force calculation passed (block 2). [Undeformed]
Momentum: 1[500000.0; 0.0; -100000.0]

Writting data file................................
Material force calculation passed (block 1). [Undeformed]
Material force calculation passed (block 2). [Undeformed]
Done
0.02%
Material force calculation passed (block 1). [Undeformed]
Material force calculation passed (block 2). [Undeformed]
0.04%
Material force calculation passed (block 1). [Undeformed]
Material force calculation passed (block 2). [Undeformed]
0.06%
Material force calculation passed (block 1). [Undeformed]
Material force calculation passed (block 2). [Undeformed]
0.08%
Material force calculation passed (block 1). [U

LoadError: InterruptException: